# SCI-3000 Development Notebook

In [ ]:
import pandas as pd
import json
from utils.SCI3000Extractor import extract_figures_and_captions
import os
from pathlib import Path
from PIL import Image
import shutil
from pathlib import Path
from tqdm.notebook import tqdm

In [4]:
folder = '../../data_links/data/SCI-3000'
annotations_folder = f'{folder}/Annotations'
pdf_folder = f'{folder}/PDFs'
target_folder = f'{folder}/Images'
sci_page_metadata_file = f'{folder}/sci-3000-page-metadata.parquet'
sci_pdf_metadata_file = f'{folder}/sci-3000-pdf-metadata.parquet'

# read parquet file
sci_page_metadata = pd.read_parquet(sci_page_metadata_file)
sci_pdf_metadata = pd.read_parquet(sci_pdf_metadata_file)

## 1 Figure extraction from PDFs

In [5]:
sci_pdf_metadata.sort_values('id').shape

(3000, 18)

In [ ]:
#list all pdfs in PDF folder
pdf_files = os.listdir(pdf_folder)
# are all pdf files from metadata in the folder?
all_pdfs_in_folder = all(f"{row['id']}.pdf" in pdf_files for index, row in sci_pdf_metadata.iterrows())
all_pdfs_in_folder

True

In [7]:
sci_pdf_metadata.head()

,id,title,year,author,subjects,lcc_code,lcc_term,lcc_2nd_lvl_code,lcc_2nd_lvl_term,url,regex_pattern,pattern_name,figures,tables,captions,references,page_count,empty_page_count
0,000361855c304169bdf874a7be8ff192,Removal of Microcystin-LR by a Novel Native Ef...,2018,[{'affiliation': 'Department of Occupational a...,"[{'code': 'R', 'scheme': 'LCC', 'term': 'Medic...",R,Medicine,R,Medicine,http://www.mdpi.com/2072-6651/10/9/363,^R.*$,Medicine,8,0,8,8,11,7
1,00157de947d4419bab9864c4a2f109cb,Palladium on Carbon-Catalyzed Suzuki-Miyaura C...,2015,[{'affiliation': 'Laboratory of Organic Chemis...,"[{'code': 'TP1-1185', 'scheme': 'LCC', 'term':...",TP1-1185,Chemical technology,TP,Chemical technology,http://www.mdpi.com/2073-4344/5/1/18,^T.*$,Technology,0,3,3,3,8,6
2,001f53a04953451fa1cdd5a5e0bd1a51,Navigating the massive world of reddit: using ...,2015,[{'affiliation': 'Department of Computer Scien...,"[{'code': 'QA75.5-76.95', 'scheme': 'LCC', 'te...",QA75.5-76.95,Electronic computers. Computer science,QA,Mathematics,https://peerj.com/articles/cs-4.pdf,^(QA75\.5-76\.95.*)|(QA76\.75-76\.765.*)$,Computer Science,5,0,5,5,13,8
3,00288843308b4a26a4fe4e8e4ef980fb,The Impact of Efflux Pump Inhibitors on the Ac...,2017,[{'affiliation': 'Department of Pharmaceutical...,"[{'code': 'QD241-441', 'scheme': 'LCC', 'term'...",QD241-441,Organic chemistry,QD,Chemistry,http://www.mdpi.com/1420-3049/22/1/114,^QD.*$,Chemistry,1,4,5,5,12,8
4,003764ba64ab4d289c1fc73a5a8ca3ca,mTHPC-loaded extracellular vesicles outperform...,2018,"[{'affiliation': 'Université de Lorraine', 'na...","[{'code': 'RM1-950', 'scheme': 'LCC', 'term': ...",RM1-950,Therapeutics. Pharmacology,RM,Therapeutics. Pharmacology,http://dx.doi.org/10.1080/10717544.2018.1513609,^R.*$,Medicine,6,0,6,6,13,8


In [7]:
sci_page_metadata.head()

,page_id,pdf_id,figures,tables,captions,references,manually_verified
0,83f36935a4ee4bb2aa11f43cce97f7bb-04,83f36935a4ee4bb2aa11f43cce97f7bb,0,0,0,0,False
1,89150a9aa0ca4f82bf50185d9ccfade0-05,89150a9aa0ca4f82bf50185d9ccfade0,1,0,1,1,False
2,b4be29d5264743669cb3d00e73edc697-02,b4be29d5264743669cb3d00e73edc697,0,0,0,0,False
3,c977ea84788147878852ebd9a0aaae44-02,c977ea84788147878852ebd9a0aaae44,0,0,0,0,False
4,4f1f1615bfab4705921bfd17f925beb8-04,4f1f1615bfab4705921bfd17f925beb8,0,0,0,0,False


In [8]:
sci_page_metadata[sci_page_metadata['figures']>1]

,page_id,pdf_id,figures,tables,captions,references,manually_verified
12,2f2d996e0b844509a5330f728d5c8139-05,2f2d996e0b844509a5330f728d5c8139,3,0,3,3,False
17,c9d59c4d7b0d441396b59c8bd61a84aa-08,c9d59c4d7b0d441396b59c8bd61a84aa,2,0,2,2,False
22,ab0fc8b9da7e4da892920e6989c8e77d-05,ab0fc8b9da7e4da892920e6989c8e77d,2,0,2,2,False
26,34d7b539543048cdbfe6158e3467ac2a-09,34d7b539543048cdbfe6158e3467ac2a,2,0,2,2,False
30,fc59830ebd2a4f1f9646c66eca4c1cf1-08,fc59830ebd2a4f1f9646c66eca4c1cf1,4,0,4,4,True
...,...,...,...,...,...,...,...
34744,98ad90436035452583315fc20117c85a-04,98ad90436035452583315fc20117c85a,2,0,2,2,False
34748,eebb2cf358614d159527002dcee0cd60-3,eebb2cf358614d159527002dcee0cd60,2,1,3,3,False
34751,d959be5488564328b49c5a379dea67dc-14,d959be5488564328b49c5a379dea67dc,4,0,4,4,False
34765,24db3670a78f4362beaaa335156ea37b-05,24db3670a78f4362beaaa335156ea37b,2,0,2,2,False


In [9]:
output_directory = "../../dataset/raw/SCI-3000-Figures-Raw"
to_extract = sci_page_metadata[sci_page_metadata['figures']>1]['page_id']
len(to_extract)

4156

In [10]:
output = extract_figures_and_captions(to_extract, pdf_folder, annotations_folder ,output_directory)

[Info] Loading existing metadata from ../../dataset/raw/SCI-3000-Figures-Raw/extracted_figures_metadata.json...
[Info] Found 4156 already processed pages. Skipping them.
All pages have already been processed.


In [11]:
output = extract_figures_and_captions(to_extract, pdf_folder, annotations_folder ,output_directory)

[Info] Loading existing metadata from ../../dataset/raw/SCI-3000-Figures-Raw/extracted_figures_metadata.json...
[Info] Found 4156 already processed pages. Skipping them.
All pages have already been processed.


In [12]:
#read metadata json file
filename = f"{output_directory}/extracted_figures_metadata.json"
data = json.load(open(filename))
len(data)

9507

In [ ]:
#len of images in ouput directory
image_files = [f for f in os.listdir(output_directory) if f.endswith('.png')]
len(image_files)

9507

In [ ]:
#show image 008d30d95bcb495ea76960dbc41ff31a-08-fig-0.png from ouput directory

image_path = f"{output_directory}/008d30d95bcb495ea76960dbc41ff31a-08-fig-0.png"
img = Image.open(image_path)
img.show()

I0000 00:00:1765321045.806490  636466 voice_transcription.cc:58] Registering VoiceTranscriptionCapability
[636428:636470:1209/235725.898836:ERROR:google_apis/gcm/engine/registration_request.cc:291] Registration response error message: DEPRECATED_ENDPOINT


In [9]:
sci_pdf_metadata[sci_pdf_metadata['id'] == '0e12397c552f4f14ac583d3094f04214']

,id,title,year,author,subjects,lcc_code,lcc_term,lcc_2nd_lvl_code,lcc_2nd_lvl_term,url,regex_pattern,pattern_name,figures,tables,captions,references,page_count,empty_page_count
150,0e12397c552f4f14ac583d3094f04214,Secure in-vehicle Systems using Authentication,2015,"[{'affiliation': None, 'name': 'Masaya Yoshika...","[{'code': 'QA75.5-76.95', 'scheme': 'LCC', 'te...",QA75.5-76.95,Electronic computers. Computer science,QA,Mathematics,https://www.atlantis-press.com/article/25022.pdf,^(QA75\.5-76\.95.*)|(QA76\.75-76\.765.*)$,Computer Science,10,1,11,11,8,2


In [11]:
sci_page_metadata[sci_page_metadata['pdf_id'] == ' 1f70ca21bbce4a3a920996e6d28c37ba']

,page_id,pdf_id,figures,tables,captions,references,manually_verified


In [14]:
sci_page_metadata[sci_page_metadata['page_id'] == '1f70ca21bbce4a3a920996e6d28c37ba-5'] 

,page_id,pdf_id,figures,tables,captions,references,manually_verified
16656,1f70ca21bbce4a3a920996e6d28c37ba-5,1f70ca21bbce4a3a920996e6d28c37ba,2,0,2,2,True


In [ ]:
annotation_file = f"{annotations_folder}/1f70ca21bbce4a3a920996e6d28c37ba-5.json"
with open(annotation_file, 'r') as f:
    annotation_data = json.load(f)
annotation_data

{'appVersion': '0.1.3',
 'secondCounter': 13,
 'canvasHeight': 1477,
 'canvasWidth': 1004,
 'annotations': [{'type': 'Annotation',
   'body': [{'type': 'TextualBody',
     'purpose': 'img-cap-enum',
     'value': 'Figure'}],
   'target': {'source': 'Unknown',
    'selector': {'type': 'FragmentSelector',
     'conformsTo': 'http://www.w3.org/TR/media-frags/',
     'value': 'xywh=pixel:136,141,748,353'}},
   '@context': 'http://www.w3.org/ns/anno.jsonld',
   'id': '#8745249480756'},
  {'type': 'Annotation',
   'body': [{'type': 'TextualBody',
     'purpose': 'img-cap-enum',
     'value': 'Caption'},
    {'type': 'TextualBody', 'purpose': 'parent', 'value': '#8745249480756'}],
   'target': {'source': 'Unknown',
    'selector': {'type': 'FragmentSelector',
     'conformsTo': 'http://www.w3.org/TR/media-frags/',
     'value': 'xywh=pixel:121,523,768,41'}},
   '@context': 'http://www.w3.org/ns/anno.jsonld',
   'id': '#8745249480717'},
  {'type': 'Annotation',
   'body': [{'type': 'TextualBod

## 2 Move real compound Figures to 03_intermediate to label them

In [ ]:
# --- KONFIGURATION ---

# 1. Quellordner (Wo liegen die Bilder und das JSON file?)
source_dir = Path("../../dataset/raw/SCI-3000-Figures-Raw") 

# 2. Zielordner (Wo sollen die 'Golden Set' Compounds hin?)
target_dir = Path("../../dataset/03_intermediate/SCI-3000_real-compound")

# 3. Name der JSON Datei (angepasst an dein Beispiel)
json_filename = "grading_labels.json"

# --- LOGIK ---

# Zielordner erstellen
target_dir.mkdir(parents=True, exist_ok=True)
json_path = source_dir / json_filename

if not json_path.exists():
    print(f"❌ Fehler: JSON Datei nicht gefunden: {json_path}")
    print("Bitte überprüfe den Pfad in 'source_dir'.")
else:
    # JSON laden
    with open(json_path, 'r') as f:
        labels = json.load(f)

    print(f"📂 Lade Labels aus {json_filename}...")
    
    count_copied = 0
    count_skipped = 0
    
    # Durchiterieren
    for filename, data in tqdm(labels.items(), desc="Verarbeite Bilder"):
        
        # --- HIER IST DIE ÄNDERUNG ---
        # Wir holen beide Werte. Standard ist False, falls ein Key fehlt.
        is_accepted = data.get("accepted", False)
        is_compound = data.get("is_compound", False)
        
        # Bedingung: Muss accepted UND compound sein
        if is_accepted and is_compound:
            
            src_file = source_dir / filename
            dst_file = target_dir / filename
            
            if src_file.exists() and not dst_file.exists():
                try:
                    # Kopieren
                    shutil.copy2(src_file, dst_file)
                    count_copied += 1
                except Exception as e:
                    print(f"⚠️ Fehler beim Kopieren von {filename}: {e}")
            else:
                print(f"⚠️ Bilddatei nicht gefunden (obwohl im JSON): {filename}")
        else:
            # Überspringen (Single Figures, Rejected, Questionable etc.)
            count_skipped += 1

    print("-" * 30)
    print(f"🎉 Fertig!")
    print(f"✅ Kopiert (Accepted & Compound): {count_copied}")
    print(f"⏭️ Übersprungen: {count_skipped}")
    print(f"📂 Zielordner: {target_dir.resolve()}")

📂 Lade Labels aus grading_labels.json...


Verarbeite Bilder:   0%|          | 0/2325 [00:00<?, ?it/s]

⚠️ Bilddatei nicht gefunden (obwohl im JSON): 000361855c304169bdf874a7be8ff192-05-fig-0.png
⚠️ Bilddatei nicht gefunden (obwohl im JSON): 003764ba64ab4d289c1fc73a5a8ca3ca-08-fig-0.png
⚠️ Bilddatei nicht gefunden (obwohl im JSON): ffe162c5d3204b3d9093d96c576d0394-10-fig-3.png
⚠️ Bilddatei nicht gefunden (obwohl im JSON): 005d3ea162634a53a42a94e7dce54455-08-fig-0.png
⚠️ Bilddatei nicht gefunden (obwohl im JSON): 005d3ea162634a53a42a94e7dce54455-08-fig-1.png
⚠️ Bilddatei nicht gefunden (obwohl im JSON): 0064bf58f91a4b6faa83c3693306f6d7-12-fig-1.png
⚠️ Bilddatei nicht gefunden (obwohl im JSON): 00668f1404e94cb1918bb5bf19444c3c-11-fig-1.png
⚠️ Bilddatei nicht gefunden (obwohl im JSON): 007827ea8dbc462ebfb18a7a650e29e5-07-fig-0.png
⚠️ Bilddatei nicht gefunden (obwohl im JSON): 00cd6e51b3884b83961791a612b22314-04-fig-1.png
⚠️ Bilddatei nicht gefunden (obwohl im JSON): 00f7197b5b964177942ee5f9637a1abc-05-fig-0.png
⚠️ Bilddatei nicht gefunden (obwohl im JSON): 00f7197b5b964177942ee5f9637a1abc-0

## 3 Move as Singel labeled Figures to 02_assets directory to label them Chart/Illustration/...

In [2]:
import os
import json
import shutil
from pathlib import Path
from tqdm.notebook import tqdm

# --- KONFIGURATION ---

# 1. Quellordner (Wo liegen die Bilder und das JSON file?)
source_dir = Path("../../dataset/raw/SCI-3000-Figures-Raw") 

# 2. Zielordner (Wo sollen die 'Singles' hin?)
# Das wird der Input-Ordner für deinen Synthesizer
target_dir = Path("../../dataset/raw/SCI-3000-Singles")

# 3. Name der JSON Datei
json_filename = "grading_labels.json"

# --- LOGIK ---

# Zielordner erstellen
target_dir.mkdir(parents=True, exist_ok=True)
json_path = source_dir / json_filename

if not json_path.exists():
    print(f"❌ Fehler: JSON Datei nicht gefunden: {json_path}")
    print("Bitte überprüfe den Pfad in 'source_dir'.")
else:
    # JSON laden
    with open(json_path, 'r') as f:
        labels = json.load(f)

    print(f"📂 Lade Labels aus {json_filename}...")
    
    count_copied = 0
    count_skipped = 0
    
    # Durchiterieren
    for filename, data in tqdm(labels.items(), desc="Verarbeite Bilder"):
        
        # --- FILTER LOGIC ---
        is_accepted = data.get("accepted", False)
        is_compound = data.get("is_compound", False)
        
        # Bedingung: Muss ACCEPTED sein aber KEIN Compound
        if is_accepted and not is_compound:
            
            src_file = source_dir / filename
            dst_file = target_dir / filename
            
            if src_file.exists():
                try:
                    # Kopieren (shutil.copy2 behält Metadaten)
                    shutil.copy2(src_file, dst_file)
                    count_copied += 1
                except Exception as e:
                    print(f"⚠️ Fehler beim Kopieren von {filename}: {e}")
            else:
                print(f"⚠️ Bilddatei nicht gefunden (obwohl im JSON): {filename}")
        else:
            # Überspringen (Rejected oder Compounds)
            count_skipped += 1

    print("-" * 30)
    print(f"🎉 Fertig!")
    print(f"✅ Kopiert (Accepted & Single): {count_copied}")
    print(f"⏭️ Übersprungen (Compounds/Rejected): {count_skipped}")
    print(f"📂 Zielordner: {target_dir.resolve()}")

📂 Lade Labels aus grading_labels.json...


Verarbeite Bilder:   0%|          | 0/2325 [00:00<?, ?it/s]

------------------------------
🎉 Fertig!
✅ Kopiert (Accepted & Single): 1160
⏭️ Übersprungen (Compounds/Rejected): 1165
📂 Zielordner: /home/tobiasponeschtu/Documents/11818774_CompoundFigureSeparation/dataset/raw/SCI-3000-Singles


## 4 Function to relabel wrongly labeled Singles to Compound Figures!

In [ ]:
#relabel Singles to Compound
def relabel_singles_to_compound(filename: str, source_dir: Path, target_dir: Path, compound_json_path: Path):
    """Verschiebt eine Datei von Singles zu Compound und aktualisiert die zugehörigen JSON-Dateien."""

    src_file = source_dir / filename
    dst_file = target_dir / filename

    # Datei verschieben, falls möglich
    if src_file.exists() and not dst_file.exists():
        try:
            shutil.move(src_file, dst_file)
            print(f"Verschoben: {filename}")
        except Exception as e:
            print(f"Fehler beim Verschieben von {filename}: {e}")
    else:
        print(f"Datei nicht gefunden oder existiert bereits im Ziel: {filename}")

    # Entferne aus single_labels.json, falls vorhanden
    singles_json_path = source_dir / "single_labels.json"
    if singles_json_path.exists():
        try:
            with open(singles_json_path, 'r') as f:
                singles_labels = json.load(f)
            if filename in singles_labels:
                del singles_labels[filename]
                with open(singles_json_path, 'w') as f:
                    json.dump(singles_labels, f, indent=4)
                print(f"Entfernt aus single_labels.json: {filename}")
        except Exception as e:
            print(f"Fehler beim Bearbeiten von single_labels.json: {e}")
    else:
        print(f"single_labels.json nicht gefunden im {source_dir}")

    # Füge zu grading_labels.json hinzu/ändere, falls vorhanden
    if compound_json_path.exists():
        try:
            with open(compound_json_path, 'r') as f:
                compound_labels = json.load(f)
            compound_labels[filename] = {"accepted": True, "is_compound": True}
            with open(compound_json_path, 'w') as f:
                json.dump(compound_labels, f, indent=4)
            print(f"Hinzugefügt zu grading_labels.json: {filename}")
        except Exception as e:
            print(f"Fehler beim Bearbeiten von grading_labels.json: {e}")
    else:
        print(f"grading_labels.json nicht gefunden: {compound_json_path}")

In [8]:
relabel_singles_to_compound('0756e60ac6724caeb22f056203135d17-09-fig-0.png', Path("../../dataset/raw/SCI-3000-Singles"), Path("../../dataset/to_label/SCI-3000"), Path("../../dataset/raw/SCI-3000-Figures-Raw/grading_labels.json"))

Datei nicht gefunden oder existiert bereits im Ziel: 0756e60ac6724caeb22f056203135d17-09-fig-0.png
Hinzugefügt zu grading_labels.json: 0756e60ac6724caeb22f056203135d17-09-fig-0.png


In [9]:
relabel_singles_to_compound('159086d0cd5e461cb8f02ff41c635676-3-fig-0.png', Path("../../dataset/raw/SCI-3000-Singles"), Path("../../dataset/to_label/SCI-3000"), Path("../../dataset/raw/SCI-3000-Figures-Raw/grading_labels.json"))

Verschoben: 159086d0cd5e461cb8f02ff41c635676-3-fig-0.png
Entfernt aus single_labels.json: 159086d0cd5e461cb8f02ff41c635676-3-fig-0.png
Hinzugefügt zu grading_labels.json: 159086d0cd5e461cb8f02ff41c635676-3-fig-0.png


In [10]:
relabel_singles_to_compound('159086d0cd5e461cb8f02ff41c635676-3-fig-1.png', Path("../../dataset/raw/SCI-3000-Singles"), Path("../../dataset/to_label/SCI-3000"), Path("../../dataset/raw/SCI-3000-Figures-Raw/grading_labels.json"))

Verschoben: 159086d0cd5e461cb8f02ff41c635676-3-fig-1.png
Entfernt aus single_labels.json: 159086d0cd5e461cb8f02ff41c635676-3-fig-1.png
Hinzugefügt zu grading_labels.json: 159086d0cd5e461cb8f02ff41c635676-3-fig-1.png


In [ ]:
relabel_singles_to_compound('21c1b78f77884755a52210307d3f43c3-2-fig-1.png', Path("../../dataset/raw/SCI-3000-Singles"), Path("../../dataset/to_label/SCI-3000"), Path("../../dataset/raw/SCI-3000-Figures-Raw/grading_labels.json"))

Verschoben: 21c1b78f77884755a52210307d3f43c3-2-fig-1.png
Entfernt aus single_labels.json: 21c1b78f77884755a52210307d3f43c3-2-fig-1.png
Hinzugefügt zu grading_labels.json: 21c1b78f77884755a52210307d3f43c3-2-fig-1.png


In [ ]:
relabel_singles_to_compound('21c1b78f77884755a52210307d3f43c3-3-fig-0.png', Path("../../dataset/raw/SCI-3000-Singles"), Path("../../dataset/to_label/SCI-3000"), Path("../../dataset/raw/SCI-3000-Figures-Raw/grading_labels.json"))

Verschoben: 21c1b78f77884755a52210307d3f43c3-3-fig-0.png
Entfernt aus single_labels.json: 21c1b78f77884755a52210307d3f43c3-3-fig-0.png
Hinzugefügt zu grading_labels.json: 21c1b78f77884755a52210307d3f43c3-3-fig-0.png


In [13]:
relabel_singles_to_compound('21c1b78f77884755a52210307d3f43c3-3-fig-1.png', Path("../../dataset/raw/SCI-3000-Singles"), Path("../../dataset/to_label/SCI-3000"), Path("../../dataset/raw/SCI-3000-Figures-Raw/grading_labels.json"))

Verschoben: 21c1b78f77884755a52210307d3f43c3-3-fig-1.png
Entfernt aus single_labels.json: 21c1b78f77884755a52210307d3f43c3-3-fig-1.png
Hinzugefügt zu grading_labels.json: 21c1b78f77884755a52210307d3f43c3-3-fig-1.png


In [14]:
relabel_singles_to_compound('27e867898645494ab7132cd4d06b2c69-17-fig-2.png', Path("../../dataset/raw/SCI-3000-Singles"), Path("../../dataset/to_label/SCI-3000"), Path("../../dataset/raw/SCI-3000-Figures-Raw/grading_labels.json"))

Verschoben: 27e867898645494ab7132cd4d06b2c69-17-fig-2.png
Entfernt aus single_labels.json: 27e867898645494ab7132cd4d06b2c69-17-fig-2.png
Hinzugefügt zu grading_labels.json: 27e867898645494ab7132cd4d06b2c69-17-fig-2.png


In [15]:
relabel_singles_to_compound('2c046ee5fd9a4f0c834d646fafc4f7dd-04-fig-0.png', Path("../../dataset/raw/SCI-3000-Singles"), Path("../../dataset/to_label/SCI-3000"), Path("../../dataset/raw/SCI-3000-Figures-Raw/grading_labels.json"))

Verschoben: 2c046ee5fd9a4f0c834d646fafc4f7dd-04-fig-0.png
Entfernt aus single_labels.json: 2c046ee5fd9a4f0c834d646fafc4f7dd-04-fig-0.png
Hinzugefügt zu grading_labels.json: 2c046ee5fd9a4f0c834d646fafc4f7dd-04-fig-0.png


In [16]:
relabel_singles_to_compound('342470416b504bf9958b6bc62bee2c99-15-fig-1.png', Path("../../dataset/raw/SCI-3000-Singles"), Path("../../dataset/to_label/SCI-3000"), Path("../../dataset/raw/SCI-3000-Figures-Raw/grading_labels.json"))

Verschoben: 342470416b504bf9958b6bc62bee2c99-15-fig-1.png
Entfernt aus single_labels.json: 342470416b504bf9958b6bc62bee2c99-15-fig-1.png
Hinzugefügt zu grading_labels.json: 342470416b504bf9958b6bc62bee2c99-15-fig-1.png


In [17]:
relabel_singles_to_compound('35949275714a4d398a3ba9a3461e2fdf-3-fig-1.png', Path("../../dataset/raw/SCI-3000-Singles"), Path("../../dataset/to_label/SCI-3000"), Path("../../dataset/raw/SCI-3000-Figures-Raw/grading_labels.json"))

Verschoben: 35949275714a4d398a3ba9a3461e2fdf-3-fig-1.png
Entfernt aus single_labels.json: 35949275714a4d398a3ba9a3461e2fdf-3-fig-1.png
Hinzugefügt zu grading_labels.json: 35949275714a4d398a3ba9a3461e2fdf-3-fig-1.png


In [18]:
relabel_singles_to_compound('3765c23ec2eb4650b3f042cdd3281df2-06-fig-0.png', Path("../../dataset/raw/SCI-3000-Singles"), Path("../../dataset/to_label/SCI-3000"), Path("../../dataset/raw/SCI-3000-Figures-Raw/grading_labels.json"))

Verschoben: 3765c23ec2eb4650b3f042cdd3281df2-06-fig-0.png
Entfernt aus single_labels.json: 3765c23ec2eb4650b3f042cdd3281df2-06-fig-0.png
Hinzugefügt zu grading_labels.json: 3765c23ec2eb4650b3f042cdd3281df2-06-fig-0.png


In [19]:
relabel_singles_to_compound('3765c23ec2eb4650b3f042cdd3281df2-06-fig-1.png', Path("../../dataset/raw/SCI-3000-Singles"), Path("../../dataset/to_label/SCI-3000"), Path("../../dataset/raw/SCI-3000-Figures-Raw/grading_labels.json"))

Verschoben: 3765c23ec2eb4650b3f042cdd3281df2-06-fig-1.png
Entfernt aus single_labels.json: 3765c23ec2eb4650b3f042cdd3281df2-06-fig-1.png
Hinzugefügt zu grading_labels.json: 3765c23ec2eb4650b3f042cdd3281df2-06-fig-1.png


In [20]:
relabel_singles_to_compound('3765c23ec2eb4650b3f042cdd3281df2-08-fig-0.png', Path("../../dataset/raw/SCI-3000-Singles"), Path("../../dataset/to_label/SCI-3000"), Path("../../dataset/raw/SCI-3000-Figures-Raw/grading_labels.json"))

Verschoben: 3765c23ec2eb4650b3f042cdd3281df2-08-fig-0.png
Entfernt aus single_labels.json: 3765c23ec2eb4650b3f042cdd3281df2-08-fig-0.png
Hinzugefügt zu grading_labels.json: 3765c23ec2eb4650b3f042cdd3281df2-08-fig-0.png


In [21]:
relabel_singles_to_compound('3a51e0ff5791428b8bee3d07b28ca20f-4-fig-0.png', Path("../../dataset/raw/SCI-3000-Singles"), Path("../../dataset/to_label/SCI-3000"), Path("../../dataset/raw/SCI-3000-Figures-Raw/grading_labels.json"))

Verschoben: 3a51e0ff5791428b8bee3d07b28ca20f-4-fig-0.png
Entfernt aus single_labels.json: 3a51e0ff5791428b8bee3d07b28ca20f-4-fig-0.png
Hinzugefügt zu grading_labels.json: 3a51e0ff5791428b8bee3d07b28ca20f-4-fig-0.png
